# 项目P3：整理 OpenStreetMap 数据

## 地图数据

[OpenStreetMap](https://www.openstreetmap.org/) 是一个开放的地图数据集，该数据包含了节点（nodes）、途径（ways）和相互关系（relations）这些主要信息，详情可参见其[维基页面](https://wiki.openstreetmap.org/wiki/OSM_XML)。


本项目选取了[北京地区](https://www.openstreetmap.org/relation/912940#map=8/40.255/116.463)的地图数据，从 [Mapzen](https://mapzen.com/data/metro-extracts/metro/beijing_china/)中直接下载而来。之所以选北京地区，是因为其数据量较其他城市更丰富，而且更为大家熟悉。

这篇报告是在 Jupyter Notebook 中完成的。首先让我们导入相应的工具包，其中 clean.py 和 osm2csv.py 是自定义的 Python 代码模块。 clean.py 负责清洗数据，osm2csv.py 将 osm 格式的地图数据进行清洗后转换成 csv 格式。


In [1]:
import pandas as pd
import re
import xml.etree.cElementTree as ET
import clean
import osm2csv

要处理的数据文件是 beijing_china.osm 。

In [2]:
OSMFILE = 'beijing_china.osm'

## 地图数据中的问题

在观察数据以后，有以下几类数据问题需要处理：
* 电话号码格式混乱（比如 +86 10 6582 2892， (010)64629112，+86-10-60712288）。
* 存在错误的邮政编码（比如位数不对或者不在北京地区范围内）。
* 营业时间格式混乱不统一（比如 10am - 11pm、9:00 - 22:00、24小时、24h等）。
* 门牌号码错误（比如门牌号中没有包含数字）。

下面我们将会逐一处理这些问题，为了表述的简洁性，这里已将数据清洗的代码块存储到 clean.py 文件中，在 notebook 中只需导入clean模块，并调用相应的处理函数即可。

### 清洗电话号码

我们发现数据中的电话号码格式不尽相同，所以需要将他们的格式进行统一。这里我们规定统一的电话格式为：“国家编号（+86） + [区号（10）] + 号码”，对于手机号、400电话，则不需要使用区号。

首先我们来查看下都有哪些电话号码不符合标准，以下列出的是 clean.py 中审查电话号码的函数，它用于查找 osm 文件中不标准的号码。

In [ ]:
# clean.py 中审查电话号码是否符合标准的函数
def audit_phone(filename): 
    '''审查文件中的电话号码是否符合标准，返回不符合标准的号码'''
    
    osm_file = open(filename, 'r')
    wrong_numbers = []   # 记录不符合标准的号码
    
    for event, elem in ET.iterparse(osm_file):
    
        if  elem.tag == 'tag':   # 获取tag的元素
            if elem.attrib['k'] in ['phone', 'contact:phone']:
                phone_number = elem.attrib['v']
                if not is_phone_standard(phone_number):  # 判断是否符合标准
                    wrong_numbers.append(phone_number)

    return wrong_numbers

通过调用 `clean.audit_phone()` 函数，我们来查看若干个不标准的号码形式。

In [3]:
wrong_number = clean.audit_phone(OSMFILE)
wrong_number[:5]

['+86 10 6582 2892',
 '(010)64629112',
 '01051696505',
 '+86-10-60712288',
 '68716285;62555813']

以下列出了 clean.py 代码中关于手机号码清洗的主要函数，分别对固定电话、手机电话和400电话进行了格式统一。

In [ ]:
def update_phone_number(phone_value):
    '''清洗单个电话号码，将不符合标准格式的电话号码转换成标准格式，不能转换的返回空字符。
    标准格式定义成：国家编号 + [区号] + 号码。'''

    # 去除非数字的字符
    digit_value = ''
    for char in phone_value:
        if char.isdigit():
            digit_value += char
    
    # 定义一些可能出现的号码格式
    # 以下列表中每一个元素都是数据中出现的号码格式，元组的第一个元素代表打头的数字，第二个元素代表位数
    phone_style = [('8610',12), ('86010', 13), ('008610', 14), ('010', 11), 
                    ('10', 10), ('86', 10), ('', 8) ]
    mobile_style = [('86', 13), ('0086', 15), ('', 11)]
    special_style = [('86400', 12), ('400', 10)]
        
    # 按固定电话、移动电话、400电话三种形式分别进行电话号码格式的标准化    
    if (digit_value[:-8], len(digit_value)) in phone_style:
        styled_value = '+86 10 ' + digit_value[-8:]
    elif ((digit_value[:-11], len(digit_value)) in mobile_style) \
            and is_mobile_phone(digit_value[-11:]):
        styled_value = '+86 ' + digit_value[-11:]
    elif (digit_value[:-7], len(digit_value)) in special_style:
        styled_value = '+86 ' + digit_value[-10:]
    else: 
        styled_value = ''  # 如果不能标准化，则返回空字符串

    return styled_value

经过清洗，之前列出的若干号码可以被统一为标准格式了，如下所示：

In [4]:
[clean.update_phone(x) for x in wrong_number[:5]]

['+86 10 65822892',
 '+86 10 64629112',
 '+86 10 51696505',
 '+86 10 60712288',
 '+86 10 68716285;+86 10 62555813']

（*注意以上列出的 `clean.update_phone_number()` 用于清洗单个号码，而 `clean.update_phone()` 可以清洗多个号码，如上面最后一项所示。*）

### 清洗邮政编码

北京的邮编都是由6位数字组成，并且以100、101、102开头。我们需要将不符合这一条件的数据找出来，并将这些错误的邮编从数据中删除。

以下是 clean.py 中用于判断邮编是否正确的函数。

In [ ]:
def is_postcode(code):
    '''判断是否符合北京邮政编码格式，是返回True，不是返回False。'''

    # 定义邮编的正则表达式，北京地区邮编以100、101、102开头，共6位数字
    postcode_parttern = r'^10[0-2]\d{3}$'
    
    return re.fullmatch(postcode_parttern, code)

通过调用 clean.py 中的 `audit_postcode()` 函数，我们来查看数据中错误的邮编。

In [5]:
clean.audit_postcode(OSMFILE)

['3208',
 '110101',
 '110023',
 '053600',
 '010-62332281',
 '10080',
 '10040',
 '10043']

这些邮编不符合北京地区的邮编格式，将在最后的清洗中被删除。

### 清洗营业时间数据

营业时间数据相对比较杂乱，这里根据可获得时间信息的丰富程度，将营业时间统一成以月份、星期、小时排列的形式，具体参见 `clean.is_hour()` 函数中给出的十种时间格式。该函数用于判断营业时间是否符合我们定义的格式，如果不符合，需要对时间进行格式统一。由于时间数据的情况比较复杂多样，具体清洗的代码可参见 clean.py 文件中的营业时间清洗部分。

In [ ]:
#clean.py 中的判断时间是否符合统一格式的函数
def is_hour(value): 
    '''判断营业时间是否满足统一的格式'''

    # 关于小时、星期、月份的正则表达式
    h = '\d{1,2}:\d{1,2}'  
    w = '(Mo|Tu|We|Th|Fr|Sa|Su)'
    m = '(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)'
    md = m +' \d{1,2}'
    
    # 定义了营业时间的统一格式，共有10中形式
    p0 = '24/7'                    # 表示7天24小时都营业
    p1 = h + '-' + h               # e.g. 06:00-23:00
    p2 = w + '-' + w + ' ' + p1    # e.g. Mo-Su 06:00-23:00
    p3 = w + ' ' + p1              # e.g. Sat 09:30-22:00
    p4 = m + '-' + m + ' ' + p2    # e.g. Apr-Oct Mo-Su 05:00-24:00
    p5 = md + '-' + md + ' ' + p1  # e.g. Apr 1-Oct 31 05:00-24:00
    p6 = p2 + ', ' + p1            # e.g. Su-Fr 08:30-11:30, 13:30-17:00
    p7 = p1 + ', ' + p1            # e.g. 08:30-11:30, 13:30-17:00
    p8 = m + '-' + m + ' ' + p1    # e.g. Apr-Oct 08:00-17:00
    p9 = p1 + ',' + p1

    # 判断数据是否满足以上给出的统一格式，是则返回True，否则返回False
    if  (re.fullmatch(p1, value) \
        or re.fullmatch(p2, value) \
        or re.fullmatch(p3, value) \
        or re.fullmatch(p4, value) \
        or re.fullmatch(p5, value) \
        or re.fullmatch(p6, value) \
        or re.fullmatch(p7, value) \
        or re.fullmatch(p8, value) \
        or re.fullmatch(p9, value) \
        or value == p0):
        return True
    else:
        return False

这里先使用 `clean.audit_hour()` 函数找出数据中不符合以上要求的时间数据。（以下列出了其中的5条。）

In [6]:
wrong_hour = clean.audit_hour(OSMFILE)
wrong_hour[-5:]

['10am-10pm',
 '11:00~12:00,17:00~19:00',
 '举办日期: 2014年10月31日到11月20日  圆顶建筑开放时间: 09:30到20:30  室外放映时间: 19:00到21:00',
 '不确定',
 '08.30AM-05.30PM']

`clean.update_hour()` 用于清洗时间数据，我们对上面所示的数据进行清洗，得到如下规范化后的时间格式。（其中第三、四条结果为空字符，表示原数据无法被统一成标准格式，会在最后的清洗过程中被剔除。）

In [7]:
[clean.update_hour(h) for h in wrong_hour[-5:]]

['10:00-22:00', '11:00-12:00,17:00-19:00', '', '', '08:30-17:30']

### 清洗门牌号码

顾名思义，门牌号需要包含数字才算正确。我们将不包含数字的门牌号认为是错误的数据，需要被清除。

下面给出的是 clean.py 中关于判断门牌号是否包含数字的函数，是则返回True，否则返回False。


In [ ]:
def is_house_number(value):
    '''如果输入值不包含数字，则认为是不正确的门牌，返回False；否则返回True。'''
    
    return re.search('\d', value)  # 是否包含数字

通过调用 clean.py 中的 `audit_house_number()` 函数，我们来查看数据中错误的门牌号。

In [8]:
clean.audit_house_number(OSMFILE)

['奥北南区',
 '北京市朝阳区',
 '北京大学计算科学与技术系',
 '八门',
 '嘉园路',
 'home guanganmen',
 '北京中联华康科技有限公司',
 'school',
 '凯隆公寓',
 '西院',
 '西院',
 '百子湾南路',
 '鲁谷大街',
 'Westin Chaoyang']

这些值都不包含数字，不符合我们认为的门牌号形式，将在最后的清洗中被删除。

## 数据清洗并存储到SQL数据库

现在我们需要执行关于以上问题的数据清洗代码，并将清洗后的数据保存到数据库中。

由于在 osm2csv.py 中的 `shape_element()` 函数中调用了 clean.py 中的清洗函数，所以 osm2csv.py 既实现了对原始数据 osm 文档的清洗，同时也将清洗后的数据存入 csv格式的文档中。

我们调用`osm2csv.process_map()` 函数将关于北京的地图的原始数据进行处理，转换成一系列csv格式的数据。

In [9]:
osm2csv.process_map(OSMFILE, validate=False)  

上述生成的一系列 csv 数据只是一个过渡文件，方便我们进一步把数据存储到 SQL 数据库中。

下面我们将 csv 文件导入到 SQL 数据库中，这里使用的是 SQLite 数据库引擎。

以下代码创建名为 openstreet 的数据库，并定义了关于节点和途径信息的五张数据表。

In [10]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column, String, Integer, Float 

# 创建数据库
engine = create_engine("sqlite:///openstreet.sqlite") 
metadata = MetaData()
                       
# 定义数据库表

node_table = Table('nodes', metadata,
                   Column('index', Integer, primary_key=True, autoincrement=True),
                   Column('id', Integer),
                   Column('lat', Float),
                   Column('lon', Float),
                   Column('user', String),
                   Column('uid', Integer),
                   Column('version', String),
                   Column('changeset', Integer),
                   Column('timestamp', String)
                  )
                          
                       
nodetag_table = Table('nodes_tags', metadata,
                      Column('index', Integer, primary_key=True, autoincrement=True),
                      Column('id', Integer),
                      Column('key', String),
                      Column('value', String),
                      Column('type', String)
                     )

                          
way_table = Table('ways', metadata,
                   Column('index', Integer, primary_key=True, autoincrement=True),
                   Column('id', Integer),
                   Column('user', String),
                   Column('uid', Integer),
                   Column('version', String),
                   Column('changeset', Integer),
                   Column('timestamp', String)
                 )                     
                
waynode_table = Table('ways_nodes', metadata,
                      Column('index', Integer, primary_key=True, autoincrement=True),
                      Column('id', Integer),
                      Column('node_id', Integer),
                      Column('position', Integer)
                     )                          
                          
waytag_table = Table('ways_tags', metadata,
                     Column('index', Integer, primary_key=True, autoincrement=True),
                     Column('id', Integer),
                     Column('key', String),
                     Column('value', String),
                     Column('type', String)
                    )

                                      
# 创建数据表
metadata.create_all(engine)



下方定义的 `csv2db()` 函数，作用是将 csv 文件中的数据导入到数据库表中。

In [11]:
def csv2db(file, engine, table): 
    
    from sqlalchemy import insert
    
    data = pd.read_csv(file, dtype=str)
    data_list = list(data.apply(dict, axis=1))
    
    connection = engine.connect()
    stmt = insert(table)
    results = connection.execute(stmt, data_list)
    connection.close()


调用 `csv2db()` 函数，将5份关于节点和途径的csv文件数据导入到相应的数据表中。至此，地图数据导入数据库的工作完成。

In [12]:
csv2db('nodes.csv', engine, node_table)
csv2db('nodes_tags.csv', engine, nodetag_table)
csv2db('ways.csv', engine, way_table)
csv2db('ways_nodes.csv', engine, waynode_table)
csv2db('ways_tags.csv', engine, waytag_table)

## 数据概述

### 数据文件的大小
通过调用Linux命令，我们可以查看原始数据，中间过程的csv文件，以及新建的数据库。原始数据 beijing_china.osm 文件有175M，数据库 openstreet.sqlite 大小为96M。其他以 .csv 结尾的文件中，nodes.csv 和 nodes_tags.csv 是关于节点的数据文件；ways.csv、ways_nodes.csv 和 ways_tags.csv 是关于途径的数据文件。

In [13]:
!du -h *.osm *.sqlite *.csv

175M	beijing_china.osm
 18M	sample_beijing_china.osm
 96M	openstreet.sqlite
 65M	nodes.csv
2.9M	nodes_tags.csv
6.9M	ways.csv
 23M	ways_nodes.csv
8.0M	ways_tags.csv


### 关于节点和途径的统计量

#### 节点（nodes）的数量

In [14]:
query = 'SELECT COUNT(*) FROM nodes'
pd.read_sql_query(query, engine)

,COUNT(*)
0,823210


####  旅馆的数量

考虑特殊的节点，比如旅馆，在nodes_tags表中需要满足 key='tourism'以及value='hotel'。

In [15]:
query = "SELECT COUNT(id) FROM nodes_tags WHERE key='tourism' AND value='hotel'"
pd.read_sql_query(query, engine)

,COUNT(id)
0,343


####  途径（ways）的数量

In [16]:
query = 'SELECT COUNT(*) FROM ways'
pd.read_sql_query(query, engine)

,COUNT(*)
0,122904


####  高架桥的数量

考虑特殊的途径，比如高架桥，在wyas_tags表中需要满足条件key='bridge'和value='viaduct'。

In [17]:
query = "SELECT count(id) FROM ways_tags WHERE key='bridge' AND value='viaduct'"
pd.read_sql_query(query, engine)

,count(id)
0,165


### 关于用户的统计量

#### 唯一用户的数量

这里需要同时统计nodes和ways这两张表。

In [18]:
query = '''SELECT COUNT(DISTINCT uid) 
FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways)'''
pd.read_sql_query(query, engine)

,COUNT(DISTINCT uid)
0,1720


#### 贡献数据量前十的用户

In [19]:
query = '''SELECT user, COUNT(user) AS num 
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) 
GROUP BY user 
ORDER BY num DESC 
LIMIT 10'''
pd.read_sql_query(query, engine)

,user,num
0,Chen Jia,213003
1,R438,143481
2,hanchao,66897
3,ij_,51905
4,Алекс Мок,47725
5,katpatuka,23621
6,m17design,21699
7,Esperanza36,18475
8,nuklearerWintersturm,16502
9,RationalTangle,13768


### 信号灯最多的途径

利用ways_nodes表可以将途径和节点关联起来，这样就可以知道每条途径上的节点情况，再联合查询nodes_tags表，就能筛选出每条途径上的特殊节点。这里将ways_nodes和nodes_tags这两张表通过ways_nodes.node_id=nodes_tags.id关联起来。

下面查询的是信号灯最多的前10条途径。

In [20]:
query = '''SELECT ways_nodes.id, COUNT(ways_nodes.id) AS sig_num 
FROM ways_nodes JOIN nodes_tags ON ways_nodes.node_id=nodes_tags.id
WHERE nodes_tags.value='traffic_signals'
GROUP BY ways_nodes.id
ORDER BY sig_num DESC
LIMIT 10'''
pd.read_sql_query(query, engine)

,id,sig_num
0,251324796,23
1,237581600,22
2,145291213,21
3,145291215,21
4,159791637,21
5,127156384,19
6,187044296,18
7,187044304,18
8,31757284,17
9,154855573,17


以上给出的是途径id和该途径上的信号灯数量，如果想要同时知道该途径的具体名称，则还要联合ways_tags表来查询。这里将上面的查询结果作为一个新的表e，和ways_tags表进行联合查询。

In [21]:
query = '''SELECT ways_tags.id, ways_tags.value, e.sig_num
FROM ways_tags JOIN 
(SELECT ways_nodes.id, COUNT(ways_nodes.id) AS sig_num 
FROM ways_nodes JOIN nodes_tags ON ways_nodes.node_id=nodes_tags.id
WHERE nodes_tags.value='traffic_signals'
GROUP BY ways_nodes.id
ORDER BY sig_num DESC
LIMIT 10) e
ON ways_tags.id=e.id
WHERE ways_tags.key='name'
ORDER BY e.sig_num DESC
'''
pd.read_sql_query(query, engine)

,id,value,sig_num
0,251324796,北土城东路,23
1,237581600,北土城东路,22
2,145291213,大屯路,21
3,145291215,大屯路,21
4,159791637,北土城东路,21
5,127156384,朝阳路,19
6,187044296,朝阳路,18
7,187044304,朝阳路,18
8,31757284,朝阳路,17
9,154855573,石景山路,17


我们注意到结果中不同id的途径具有相同的名称（value）值，这大概是由于一条道路需要由多条途径（ways）组成。

## 关于数据集的其他想法

### 数据标注的完整性讨论

我们以节点（nodes）数据为例，通过下面的查询发现，总的节点数为823210个，而被标记（也就是具有tag属性）的节点数据只有40812个，占了不到总数的5%。可见，目前北京地区的地图信息还是不够全面丰富的，有待更进一步的补充和完善。

In [22]:
# 总节点数
query = 'SELECT COUNT(DISTINCT id) FROM nodes'
nodes_num = pd.read_sql_query(query, engine).iloc[0,0]
nodes_num

823210

In [23]:
# 被标注的节点数
query = 'SELECT COUNT(DISTINCT id) FROM nodes_tags'
taged_nodes_num = pd.read_sql_query(query, engine).iloc[0,0]
taged_nodes_num

40812

In [24]:
# 标注节点数占总节点数的比例
taged_nodes_num / nodes_num

0.049576657232055003

完善地图数据的标注信息是一个非常有挑战性的问题，但同时也会提供更丰富的数据分析维度。如何增加数据的标注信息，这里给出若干个建议：

**建议1：** 用奖励的方式提高大家贡献数据的积极性。

由于OpenStreetMap是一个开放的可由大家共同编辑的地图数据集，所以需要增加大家贡献数据的积极性，让更多的人愿意参与进来，比如可以通过节点的重要性来设置奖励等级等一系列互动好玩的方法增强积极性。

*好处：*
1. 通过奖励，提高大家对贡献数据的热情，快速累积大量数据。
2. 提高重要节点的奖励额度，还可以优先收集重要信息。

*预期的问题：*
1. 人工添加的数据往往存在各种错误，准确性不能保障。
2. 不排除有人为了获取奖励而胡乱编造数据。

**建议2：** 通过智能机器采集地理数据。

除了由人力众筹的方式贡献数据，还可以采用机器，从网络或通过地理采集车等其他途径采集相关的数据。

*好处：*机器采集数据的效率高，且相对人工更准确。

*预期的问题：*
1. 机器采集的成本相对较大。
2. 有些地理数据是机器无法获取的。


**建议3：** 增加数据可信度等级标记。

当然，在增加标注信息的时候，也要注意数据的准确性，数据中可以增加数据可信度等级，当有更多人对这一信息认可时它的可信度等级就越高。

*好处：*通过可信度等级，评估该数据的准确性，即可防范人为的信息错误，也可防范机器采集的不准确性。

*预期的问题：*成本较高，需要较多的人力对同一数据进行打分核对。


### 数据地理分布情况的讨论。

这里我们来讨论节点数据在北京范围内的分布情况。首先我们需要找到北京中心的经纬度，并以此为中心将北京划分为东北、西北、东南、西南四个区域。然后查看每个区域内的节点数量。

以下的查询是首先找到北京市的id，并通过该id查找北京所在的经纬度，并认为这就是北京中心地点的坐标。

In [25]:
# 查找北京市的id信息
query = "SELECT * FROM nodes_tags WHERE key='name' AND value='北京市'"
pd.read_sql_query(query, engine)

,index,id,key,value,type
0,2,25248662,name,北京市,regular


In [26]:
# 查找北京市的经纬度信息
query = "SELECT * FROM nodes WHERE id=25248662"
pd.read_sql_query(query, engine)

,index,id,lat,lon,user,uid,version,changeset,timestamp
0,1,25248662,39.905963,116.391248,ff5722,3450290,87,44008181,2016-11-28T14:28:46Z


有了北京中心的坐标后，就可以根据每个id携带的经纬度数据，统计在四个不同区域内的节点数据了，并比较每个区域节点数占总节点数比例的情况。

* 在东北方向，总共有225099个节点，占总节点数的27.34%。

In [27]:
# 东北方向节点数
query = "SELECT COUNT(DISTINCT id) FROM nodes WHERE lat>39.905963 AND lon>116.391248"
east_north = pd.read_sql_query(query, engine).iloc[0,0]
east_north

225099

In [28]:
# 东北方向节点数占比
east_north / nodes_num

0.27344055587274207

* 在西北方向，总共有291253个节点，占总节点数的35.38%。

In [29]:
# 西北方向节点数
query = "SELECT COUNT(DISTINCT id) FROM nodes WHERE lat>39.905963 AND lon<116.391248"
west_north = pd.read_sql_query(query, engine).iloc[0,0]
west_north

291253

In [30]:
# 西北方向节点数占比
west_north / nodes_num

0.35380158161344005

* 在东南方向，总共有175337个节点，占总节点数的21.30%。

In [31]:
# 东南方向节点数
query = "SELECT COUNT(DISTINCT id) FROM nodes WHERE lat<39.905963 AND lon>116.391248"
east_south = pd.read_sql_query(query, engine).iloc[0,0]
east_south

175337

In [32]:
# 东南方向节点数占比
east_south / nodes_num

0.21299182468628905

* 在西南方向，总共有131520个节点，占总节点数的15.98%。

In [33]:
# 西南方向节点数
query = "SELECT COUNT(DISTINCT id) FROM nodes WHERE lat<39.905963 AND lon<116.391248"
west_south = pd.read_sql_query(query, engine).iloc[0,0]
west_south

131520

In [34]:
# 西南方向节点数占比
west_south / nodes_num

0.15976482307066242

从上面计算的数据中，我们发现处在东北、西北方向的节点是远多于东南、西南方向的节点数的。北京北部地区的节点数占了62.72%，而南部地区只占了37.28%。这大概与北京南北区域发展不平衡有关，一般来说，北京的北边的地区经济更为发达，人口更为密集，而南边地区相对来说发展的没那么好，所以导致了节点数据在地理位置上分布的不均衡。

In [35]:
# 北部地区节点占比
(east_north + west_north) / nodes_num

0.62724213748618218

In [36]:
# 南部地区节点占比
(east_south + west_south) / nodes_num

0.37275664775695144

以上关于数据在地理分布的上的讨论还是比较粗浅的，只是粗暴的把北京地区分为四个大致的区域。如果能根据节点所在城区进行分析，将更有价值，但是数据中缺乏城区的信息，所以对数据集提出以下建议。

**建议：** 数据中需要增加节点或途经所在城区（比如海淀区、朝阳区等）的信息。

*好处：*
1. 按城区划分地理位置进行讨论，比上述使用粗略的东南西北方向划分更有意义。
2. 有了城区数据，可以扩展数据分析的维度，让关于数据地理分布的讨论更完善。
3. 可以分析各个城区的商店、银行、道路情况等信息，来观察各城区的经济发展水平。

*预期的问题：*
1. 需要考虑增加城区信息的成本。（如果用人工，比较费时费力；考虑是否可以通过经纬度来确定所在城区。）
2. 数据中的途径（way）可能跨越两个城区，对于这样的情况，需要仔细对待。




## 总结

这篇报告整理并分析了OpenStreetMap开放地图中关于北京地区的数据。
* 首先，是对数据做了一些清洗工作，比如对电话号码、营业时间格式做了统一，并剔除了错误的邮政编码和门牌号数据。

* 然后，将清洗后的数据存入SQLite数据库中，并通过SQL查询语句统计了数据中的若干信息，比如节点和途径的相关统计、用户的相关统计，以及查看了信号灯最多的前十条途径和它们的名称。

* 最后，给出了关于数据集的一些其他想法，这里讨论了数据标注的完整性问题，以及节点在地理分布上的差异情况。

## 参考资源

* [OpenStreetMap官网](https://www.openstreetmap.org/)
* [OpenStreetMap 维基页面](https://wiki.openstreetmap.org/wiki/Main_Page)
* [Mapzen 北京地区数据下载地址](https://mapzen.com/data/metro-extracts/metro/beijing_china/)
* [手机号码百度百科](https://baike.baidu.com/item/手机号码)
* [北京邮编区号大全](http://www.ip138.com/post/beijing/)